<a href="https://colab.research.google.com/github/KazDev17/Anime-Success-Predictor/blob/main/Anime_Rating_Predictor_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Project Introduction

Anime Success Predictor: Deep Learning Regression
This project utilizes a dataset of over 14,000 anime entries to predict user ratings on MyAnimeList. Using a Deep Neural Network built with TensorFlow, we analyze metadata like Studios, Genres, Themes, and Source material to understand what drives high-rated content.

2. Data Cleaning & Safety



Phase 2: Data Cleaning & Imputation
To ensure model stability, I removed entries missing a 'Score' and perform Imputation on missing categorical data.

By filling missing 'Themes' with "None", we retain 40% more data than a standard row-deletion approach.

In [ ]:
import pandas as pd
# df = pd.read_csv('mal_anime.csv')
FILE_ID = '1qR68BPtRuQ6BRuv-uBoF5Eya9bn0P_Ws'
data_url = f'https://drive.google.com/uc?export=download&id=1qR68BPtRuQ6BRuv-uBoF5Eya9bn0P_Ws'

try:
    df = pd.read_csv(data_url)
    print(f"✅ Dataset loaded successfully! Rows: {len(df)}")
except Exception as e:
    print(f"❌ Error loading data: {e}")
    print("Check if the Google Drive link is set to 'Anyone with the link'.")

# Select features
useful_columns = ['Type', 'Episodes', 'Source', 'Genres', 'Themes', 'Studios', 'Released_Year', 'Score']
df_clean = df[useful_columns].copy()

# Critical Cleaning
df_clean = df_clean.dropna(subset=['Score']) # Remove rows without a target
df_clean['Themes'] = df_clean['Themes'].fillna('None') # Impute missing themes
df_clean['Episodes'] = pd.to_numeric(df_clean['Episodes'], errors='coerce').fillna(1)
df_clean['Released_Year'] = df_clean['Released_Year'].fillna(df_clean['Released_Year'].median())
df_clean = df_clean.dropna(subset=['Genres', 'Studios']) # Remove rows with no clues

print(f"Dataset Cleaned: {len(df_clean)} samples ready for training.")

3. Feature Engineering

Phase 3: Feature Engineering (One-Hot Encoding)

Neural Networks require numerical input. I use One-Hot Encoding to transform categorical strings into a high-dimensional binary matrix. This results in nearly 1,000 unique features for the model to analyze.

In [ ]:
# Convert text to binary features
X = df_clean[['Released_Year', 'Episodes']].copy()
X['Main_Type'] = df_clean['Type']
X['Main_Source'] = df_clean['Source']
X['Main_Genre'] = df_clean['Genres'].str.split(',').str[0]
X['Main_Studio'] = df_clean['Studios'].str.split(',').str[0]
X['Main_Theme'] = df_clean['Themes'].str.split(',').str[0]

X = pd.get_dummies(X, columns=['Main_Type', 'Main_Source', 'Main_Genre', 'Main_Studio', 'Main_Theme'])
y = df_clean['Score']

# Split and Scale
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

4. Model Training

Phase 4: Building the Neural Network

We implement a 4-layer Sequential model.

We include a Dropout layer (20%) to prevent overfitting, ensuring the model generalizes well to anime released in the future.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2, verbose=1)

model.summary()

5. Results & Visualizations

Phase 5: Performance Evaluation

To evaluate the model, we plot the training and validation loss. \

This allows us to visualize the learning curve and identify if the model has successfully converged or if overfitting has occurred.

In [ ]:
import matplotlib.pyplot as plt

# Set up the plotting area
plt.figure(figsize=(14, 5))

# Plot 1: Mean Absolute Error (Accuracy)
plt.subplot(1, 2, 1)
plt.plot(history.history['mae'], label='Training MAE', color='#1f77b4', lw=2)
plt.plot(history.history['val_mae'], label='Validation MAE', color='#ff7f0e', lw=2)
plt.title('Model Accuracy (MAE)')
plt.xlabel('Epochs')
plt.ylabel('Error (MAL Points)')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Loss (Mean Squared Error)
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss', color='#1f77b4', lw=2)
plt.plot(history.history['val_loss'], label='Validation Loss', color='#ff7f0e', lw=2)
plt.title('Model Loss (MSE)')
plt.xlabel('Epochs')
plt.ylabel('Loss Score')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Final Evaluation on the 20% "Final Exam" data
final_results = model.evaluate(X_test, y_test, verbose=0)
print(f"Final Test MAE: {final_results[1]:.4f}")

6. Interactive Prediction Demo

Phase 6: Interactive "What-If" Analysis
Use the interface below to test hypothetical anime configurations.

This demo demonstrates the model's ability to perform Inference on user-provided data by mapping text inputs to the trained 995-feature vector.

In [ ]:
# --- 1. PREDICTION FUNCTION ---
import numpy as np

def predict_my_anime(type_name, source, genre, studio, theme, year, episodes):
    # Create an empty row matching the training features
    input_data = pd.DataFrame(np.zeros((1, X.shape[1])), columns=X.columns)

    # Fill numeric data
    input_data['Released_Year'] = year
    input_data['Episodes'] = episodes

    # Map categorical inputs to One-Hot columns
    if f'Main_Type_{type_name}' in input_data.columns:
        input_data[f'Main_Type_{type_name}'] = 1
    if f'Main_Source_{source}' in input_data.columns:
        input_data[f'Main_Source_{source}'] = 1
    if f'Main_Genre_{genre}' in input_data.columns:
        input_data[f'Main_Genre_{genre}'] = 1
    if f'Main_Studio_{studio}' in input_data.columns:
        input_data[f'Main_Studio_{studio}'] = 1
    if f'Main_Theme_{theme}' in input_data.columns:
        input_data[f'Main_Theme_{theme}'] = 1

    # Scale the input using the same parameters as the training data
    input_scaled = scaler.transform(input_data)

    # Predict
    prediction = model.predict(input_scaled, verbose=0)
    return prediction[0][0]

# --- 2. INTERACTIVE UI ---
# @title 📺 Ultimate Anime Score Predictor
# Fill these out to test your most niche anime ideas!

Type = "TV" # @param ["TV", "Movie", "OVA", "ONA", "Special", "Music"]
Source = "Manga" # @param ["Manga", "Original", "Light Novel", "Visual Novel", "Web manga", "Novel", "Game", "Other"]
Genre = "Fantasy" # @param ["Action", "Adventure", "Avant Garde", "Award Winning", "Boys Love", "Comedy", "Drama", "Ecchi", "Erotica", "Fantasy", "Girls Love", "Gourmet", "Horror", "Mystery", "Romance", "Sci-Fi", "Slice of Life", "Sports", "Supernatural", "Suspense"]
Theme = "Isekai" # @param ["Adult Cast", "Anthropomorphic", "CGDCT", "Childcare", "Combat Sports", "Detective", "Educational", "Gore", "Harem", "High Stakes Game", "Historical", "Idols (Female)", "Idols (Male)", "Isekai", "Iyashikei", "Love Polygon", "Magical Sex Shift", "Mahou Shoujo", "Martial Arts", "Mecha", "Medical", "Military", "Music", "Mythology", "Organized Crime", "Otaku Culture", "Parody", "Performing Arts", "Pets", "Psychological", "Racing", "Reincarnation", "Reverse Harem", "Romantic Subtext", "Samurai", "School", "Showbiz", "Space", "Strategy Game", "Super Power", "Survival", "Team Sports", "Time Travel", "Vampires", "Video Game", "Visual Arts", "Workplace", "None"]
Studio = "TMS Entertainment" # @param ["Madhouse", "Sunrise", "Toei Animation", "Bones", "MAPPA", "A-1 Pictures", "Ufotable", "Kyoto Animation", "Wit Studio", "Shaft", "Production I.G", "J.C.Staff", "Studio Pierrot", "TMS Entertainment", "Studio Ghibli", "Shin-Ei Animation"]
Year = 2026 # @param {type:"slider", min:1960, max:2030, step:1}
Episodes = 24 # @param {type:"number"}

# Run the prediction using the function we built earlier
score = predict_my_anime(Type, Source, Genre, Studio, Theme, Year, Episodes)

print(f"--- PREDICTION RESULT ---")
print(f"Type: {Type} | Source: {Source} | Studio: {Studio}")
print(f"Genre: {Genre} | Theme: {Theme}")
print(f"Year: {Year} | Episodes: {Episodes}")
print(f"\nPredicted MAL Score: {score:.2f}")

Conclusion

Conclusion & Key Learnings


*   **Model Performance**: The model predicts anime scores with an average error of **0.48 points**

*   **Non-Linear Relationships**: I chose a Neural Network over a standard Linear Regression because the factors affecting anime popularity (like Studio reputation vs. Genre trends) are non-linear and complex.


*   **Feature Engineering**: One-Hot Encoding proved essential, turning categorical metadata into a 995-feature vector that the model could mathematically process.

*   **Future Improvements**: Adding user-review sentiment analysis or budget data could further refine the model's accuracy.